In [1]:
model = "llama3.2:1B"

#### Task 1: Simple Chain with Retrieval

**Objective:**

Implement a simple RAG chain with ChatOllama, HuggingFaceEmbeddings and Chroma.

Process:

1. Retrieve documents from chroma db based on query
2. Invoke chain with retrieved documents as input

**Task Description:**

- load llm model via ollama
- load embedding model via ollama with `ollama pull pull bge-m3` (if not yet done)
- create chroma db client
- create prompt template for summarization
- create simple chain with following steps: retrieved documents, prompt, model, output parser
- create query and perform similarity search with a query
- invoke chain and pass retrieved documents to the chain

**Useful links:**

- [RAG with Ollama](https://python.langchain.com/v0.2/docs/tutorials/local_rag/)


In [2]:
from langchain_ollama import ChatOllama

model = ChatOllama(model=model)

In [3]:
from langchain_ollama import OllamaEmbeddings

embedding_model = OllamaEmbeddings(
    model="bge-m3",
)

In [4]:
from langchain_chroma import Chroma
import chromadb
import chromadb
from chromadb.config import DEFAULT_TENANT, DEFAULT_DATABASE, Settings

client = chromadb.HttpClient(
    host="localhost",
    port=8000,
    ssl=False,
    headers=None,
    settings=Settings(allow_reset=True, anonymized_telemetry=False),
    tenant=DEFAULT_TENANT,
    database=DEFAULT_DATABASE,
)

collection = client.get_or_create_collection("ai_model_book")

vector_db_from_client = Chroma(
    client=client,
    collection_name="ai_model_book",
    embedding_function=embedding_model,
)

In [5]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    "Summarize the main themes in these retrieved docs: {docs}"
)


# Convert loaded documents into strings by concatenating their content
# and ignoring metadata
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


chain = {"docs": format_docs} | prompt | model | StrOutputParser()

In [7]:
search_query = "Types of Machine Learning Systems"

docs = vector_db_from_client.similarity_search(search_query)

print(docs)

[Document(id='4231ed16-567d-4433-ae62-b8225ab5ba65', metadata={'creationdate': '2019-05-07T15:51:31+00:00', 'creator': 'AH CSS Formatter V6.2 MR4 for Linux64 : 6.2.6.18551 (2014/09/24 15:00JST)', 'moddate': '2019-05-07T15:51:31+00:00', 'page': 33, 'page_label': '8', 'producer': 'Antenna House PDF Output Library 6.2.609 (Linux64)', 'source': 'src/AI_Book.pdf', 'title': 'Hands-On Machine Learning with Scikit-Learn and TensorFlow', 'total_pages': 510, 'trapped': '/False'}, page_content='Types of Machine Learning Systems\nThere are so many different types of Machine Learning systems that it is useful to\nclassify them in broad categories based on:\n• Whether or not they are trained with human supervision (supervised, unsuper‐\nvised, semisupervised, and Reinforcement Learning)\n• Whether or not they can learn incrementally on the fly (online versus batch\nlearning)\n• Whether they work by simply comparing new data points to known data points,\nor instead detect patterns in the training dat

In [ ]:
# Simple chain invoke
chain.invoke(docs)

"The retrieved documents discuss various types and subcategories of machine learning (ML) systems. Here's a summary of the main themes:\n\n1. **Classification of ML Systems**: The authors categorize ML systems into four major groups based on the amount and type of supervision, including:\n\t* Supervised Learning: training data with labels\n\t* Unsupervised Learning: no explicit labels or training data\n\t* Semisupervised Learning: a combination of labeled and unlabeled data\n\t* Reinforcement Learning: learning through trial and error\n\n2. **Supervision**: The importance of supervision in ML is highlighted, including the need for monitoring performance and adapting to changes in data.\n\n3. **Learning Paradigms**:\n\t* Supervised Learning: where the system learns from labeled data and generalizes to new instances\n\t* Unsupervised Learning: where the system learns from unlabeled data and finds patterns or structures\n\t* Semisupervised Learning: a combination of supervised and unsuper

In [ ]:
# Simple stream the chain output
for chunk in chain.stream(docs):
    print(chunk, end="", flush=True)

Based on the retrieved documents, the main themes in the topic "Types of Machine Learning Systems" are:

1. **Classification**: Machine learning systems can be classified into different categories based on their characteristics, such as whether they are trained with human supervision, learn incrementally, or work by detecting patterns.

2. **Supervised vs. Unsupervised Learning**: The distinction between supervised and unsupervised learning is crucial in machine learning. Supervised learning involves labeling data to train the model, while unsupervised learning involves identifying patterns in unlabeled data.

3. **Instance-Based vs. Model-Based Learning**: Instance-based learning focuses on making predictions by comparing new instances to existing ones, whereas model-based learning involves building a predictive model that can generalize to new instances.

4. **Generalization**: The ability of machine learning systems to generalize to new instances is critical in various applications,

In [ ]:
# More complex async event streaming
async for event in chain.astream_events(docs, version="v2"):
    kind = event["event"]
    if kind == "on_chat_model_stream":
        print(event["data"]["chunk"].content, end="", flush=True)

The retrieved docs provide an overview of different types of Machine Learning (ML) systems. Here are the main themes summarized:

1. **Machine Learning Systems Classification**: The docs discuss how ML systems can be classified based on four criteria:
	* Training data type (supervised, unsupervised, semisupervised, and Reinforcement Learning)
	* Learning approach (online versus batch learning)
	* Generalization method (instance-based or model-based)
2. **Supervised vs. Unsupervised Learning**: The docs explain the differences between supervised and unsupervised ML systems:
	* Supervised learning: training data includes desired solutions (labels) and is used to make predictions
	* Unsupervised learning: training data does not include labels, and algorithms learn to identify patterns in the data
3. **Instance-Based vs. Model-Based Learning**: The docs highlight two main approaches to generalization:
	* Instance-based learning: systems learn by heart, comparing new instances to previously

#### Task 2: Q&A with RAG

**Objective:**

Implement a Q/A retrieval chain with ChatOllama, HuggingFaceEmbeddings and Chroma

**Task Description:**

- create RAG-Q/A prompt template
- create retriever from vector db client (instead of manually passing in docs, we automatically retrieve them from our vector store based on the user question)
- create simple chain with following steps: retriever, formatting retrieved docs, user question, prompt, model, output parser
- create question for Q/A retrieval chain
- invoke chain and with question

**Useful links:**

- [RAG with Ollama](https://python.langchain.com/v0.2/docs/tutorials/local_rag/)


In [ ]:
from langchain_core.runnables import RunnablePassthrough

prompt_template = """
You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.

<context>
{context}
</context>

Answer the following question:

{question}"""

rag_prompt = ChatPromptTemplate.from_template(prompt_template)

retriever = vector_db_from_client.as_retriever()

qa_rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | rag_prompt
    | model
    | StrOutputParser()
)

In [17]:
qa_rag_chain

{
  context: VectorStoreRetriever(tags=['Chroma', 'OllamaEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x114b82790>, search_kwargs={})
           | RunnableLambda(format_docs),
  question: RunnablePassthrough()
}
| ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="\nYou are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\n\n<context>\n{context}\n</context>\n\nAnswer the following question:\n\n{question}"), additional_kwargs={})])
| ChatOllama(model='llama3.2:1B')
| StrOutputParser()

In [18]:
question = "What is supervised learning?"

qa_rag_chain.invoke(question)

'Supervised learning is a type of machine learning where the system learns to map inputs (features) to outputs (labels or targets) using labeled data. In other words, the training data contains both input features and corresponding labels that define the correct output. The goal of supervised learning is to create a model that can predict the label for new, unseen inputs based on the patterns learned from the labeled data.'

In [19]:
# More complex async event streaming
async for event in qa_rag_chain.astream_events(question, version="v2"):
    kind = event["event"]
    if kind == "on_chat_model_stream":
        print(event["data"]["chunk"].content, end="", flush=True)

Supervised learning is a type of machine learning where an algorithm is trained on labeled data, meaning that the correct output or label for each input is already known. The goal is to train the algorithm so it can make predictions or decisions based on new, unseen data by learning from examples with correct and incorrect outputs.

#### Alternative: Using pre-built ConversationalRetrievalChain Class

In [6]:
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

In [7]:
retriever = vector_db_from_client.as_retriever()
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

/var/folders/4_/gdhx54ks4g90mppmxc0wny5m0000gn/T/ipykernel_68637/2064999211.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


In [8]:
qa_chain = ConversationalRetrievalChain.from_llm(
    model, retriever=retriever, memory=memory, verbose=False
)

In [10]:
# More complex async event streaming
async for event in qa_chain.astream_events("What is supervised learning?", version="v2"):
    kind = event["event"]
    if kind == "on_chat_model_stream":
        print(event["data"]["chunk"].content, end="", flush=True)

Supervised learning is a type of machine learning where an algorithm is trained on labeled data, meaning that each sample or instance has a corresponding target or outcome. The goal of supervised learning is to learn a mapping between input data and output labels or classes, so that the algorithm can predict the correct label for new, unseen input data.

In other words, in supervised learning:

1. **Data is labeled**: Each sample or instance is assigned a label or output variable.
2. **Algorithm learns from data**: The algorithm is trained on the labeled data to learn patterns and relationships between inputs and outputs.
3. **Predictions are made**: Once the algorithm has learned from the data, it can make predictions on new, unseen input data by using its learned mapping.

Supervised learning algorithms typically involve the following steps:

1. **Data preprocessing**: Cleaning, transforming, and preparing the labeled data for training.
2. **Model selection**: Choosing a suitable sup

In [11]:
# More complex async event streaming
async for event in qa_chain.astream_events("Which algorithm was the last one in your list?", version="v2"):
    kind = event["event"]
    if kind == "on_chat_model_stream":
        print(event["data"]["chunk"].content, end="", flush=True)

Here's a rephrased version of the follow-up question as a standalone question:

What are some common types of supervised learning algorithms used for real-world applications?There are several common types of supervised learning algorithms used for real-world applications. Some examples include:

k-Nearest Neighbors
Linear Regression
Logistic Regression
Support Vector Machines (SVMs)
Decision Trees and Random Forests
Neural networks2
These algorithms can be broadly categorized into two main groups: instance-based learning, where the algorithm builds a model from individual data points, and model-based learning, where the algorithm builds a model that generalizes to unseen data.